In [1]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy

# Original Settings

In [2]:
elems0,lattices,labor = mli.readInputfile('mli.in.origin')

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [3]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [4]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)

In [5]:
labor = ['iotaline','mapout','fin']
mli.writeInputfile(elems,lattices,labor)
mli.run()
M,G = mli.readTransferMap()

### Quadrupoles

In [6]:
indexQuad = []
QuadStrength = []
for i,item in enumerate(elems):
    if 'quadrupole' == item.elem:
        if 'k1' in item.keys():
            indexQuad.append(i)
            QuadStrength.append(item.k1)
nQuad = len(indexQuad)

### Sextupoles

In [7]:
indexSext = []
SextStrength = []
for i,item in enumerate(elems):
    if 'sextupole' == item.elem:
        indexSext.append(i)
        SextStrength.append(item.g2)
nSext = len(indexSext)

# Optimize

In [8]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx

In [9]:
M_ref = np.array([[ 1.0, 0.0,  0.0,  0.0,  0.0],
                  [ -k , 1.0,  0.0,  0.0,  0.0],
                  [ 0.0, 0.0,  1.0,  0.0,  0.0],
                  [ 0.0, 0.0,   -k,  1.0,  0.0]])

In [10]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
G2_ref.head()

,exponents,GP
28,f( 30 00 00 ),0.000000
29,f( 21 00 00 ),0.000000
33,f( 20 00 01 ),0.727236
34,f( 12 00 00 ),0.000000
38,f( 11 00 01 ),0.000000


In [11]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2

In [12]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0

In [13]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [14]:
#%%
def objFunc(arg):     
    for i,j in enumerate(indexQuad):
        elems[j]['k1']=arg[i]
    for i,j in enumerate(indexSext):
        elems[j]['g2']=arg[i+nQuad]
        
    mli.writeInputfile(elems,lattices,labor)
    mli.run()
    
    M,G = mli.readTransferMap()
    
    obj = 1.0e36* np.sum((M.values[:4,[0,1,2,3,5]]-M_ref)**2) \
         +1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
         +1.0e12* np.sum((G.loc[210:450,'GP'].values-G4_ref['GP'].values)**2) \
         +        np.sum((G.loc[462:910,'GP'].values-G5_ref['GP'].values)**2)
    return obj

In [15]:
scale = np.zeros(len(QuadStrength)+len(SextStrength))
mid   = np.zeros(len(QuadStrength)+len(SextStrength))
i=0
for k1 in QuadStrength:
    mid[i] = k1
    scale[i] = 0.04*np.abs(k1)
    i=i+1
for g2 in SextStrength:
    mid[i] = 0.0
    scale[i] = 400.0
    i=i+1

In [16]:
import pickle
with open('result.quad.sext','rb') as fp:
    result=pickle.load(fp)

In [17]:
if hasattr(result,'x'): 
    x = result.x
else:
    x = result.population[0]
    x = (x-0.5)*scale + mid

In [18]:
print(objFunc(QuadStrength + SextStrength))
M0,G0 = mli.readTransferMap()

9.690949264855172e+31


In [19]:
M0

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [20]:
G0.head()

,exponents,GP
28,f( 30 00 00 ),7.765582e-12
29,f( 21 00 00 ),-5.701697e-12
33,f( 20 00 01 ),7.272738e-01
34,f( 12 00 00 ),-2.987872e-12
38,f( 11 00 01 ),5.580342e-12


In [21]:
print(objFunc(x))
M1,G1 = mli.readTransferMap()

2.429168000600238e+32


In [22]:
M1

,1,2,3,4,5,6
1,0.996219,0.000933,0.000000,0.000000,0.0,-0.000869
2,-1.455445,1.002433,0.000000,0.000000,0.0,-0.001122
3,0.000000,0.000000,0.997143,0.001407,0.0,0.000000
4,0.000000,0.000000,-1.452236,1.000817,0.0,0.000000
5,0.002382,-0.000870,0.000000,0.000000,1.0,-1.324572
6,0.000000,0.000000,0.000000,0.000000,0.0,1.000000


In [23]:
G1.head()

,exponents,GP
28,f( 30 00 00 ),-0.747539
29,f( 21 00 00 ),-2.452496
33,f( 20 00 01 ),-4.832963
34,f( 12 00 00 ),3.709044
38,f( 11 00 01 ),1.524600
